In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm  # one way to do regs
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# new today...


In [12]:
from statsmodels.formula.api import ols as sm_ols
# you can include the interaction of x and z by adding "+x*z" in the spec, like:

sm_ols('lprice ~ lcarat + ideal + lcarat*ideal',
       data=diamonds2).fit().summary()

#This shows that a 1% increase in carats is associated with 
    #a 1.47% increase in price for fair diamonds, 
    #but a 1.78% increase for ideal diamonds (1.47+0.28).
#Thus: The return on carats is different (and higher) for better cut diamonds!

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 lprice   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.936
Method:                 Least Squares   F-statistic:                 2.611e+05
Date:                Thu, 16 Apr 2020   Prob (F-statistic):               0.00
Time:                        09:58:02   Log-Likelihood:                -3119.8
No. Observations:               53797   AIC:                             6248.
Df Residuals:                   53793   BIC:                             6283.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                8.4154      0.002   5171.790      0.000       8.412       8.419
ideal[T.True]            0.1102      0.003     38.149      0.000       0.104       0.116
lcarat                   1.6875      0.002    687.156      0.000       1.683       1.692
lcarat:ideal[T.True]     0.0229      0.004      5.766      0.000       0.015       0.031
==============================================================================
Omnibus:                      772.974   Durbin-Watson:                   1.239
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1339.337
Skew:                           0.102   Prob(JB):                    1.47e-291
Kurtosis:                       3.746   Cond. No.                         5.33
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
from statsmodels.formula.api import ols as sm_ols


diamonds = sns.load_dataset('diamonds')
diamonds2 = (diamonds
             .query('carat<2.5')
             .assign(lprice=np.log(diamonds['price']))#create a new function
             .assign(lcarat=np.log(diamonds['carat']))
             .assign(ideal = diamonds['cut'] == 'Ideal')
             .assign(const = 1)
            )
y = diamonds2['lprice']
X = diamonds2[['const','lcarat']]

diamonds2.groupby('cut')['lprice'].mean() # avg lprice by cut

cut
Fair         8.068832
Good         7.836076
Ideal        7.636921
Premium      7.944690
Very Good    7.795675
Name: lprice, dtype: float64

In [14]:
standardize = lambda x:x/x.std() # normalize(df['x']) will divide all 'x' by the std deviation of 'x'

print("Divide lcarat by its std dev:\n")
print(sm_ols('lprice ~ lcarat + ideal', 
       # for **just** this regression, divide 
       data=diamonds2.assign(lcarat = standardize(diamonds2['lcarat'])) 
       # this doesn't change the diamonds2 data permanently, so the next time you call on
       # diamonds2, you can use lcarat as if nothing changed. if you want to repeat this
       # a bunch, you might instead create and save a permanent variable called "lcarat_std"
       # where "_std" indicates that you divided it by the std dev.
      ).fit().params)

print("\n\nThe original reg:\n")
print(sm_ols('lprice ~ lcarat + ideal',data=diamonds2 ).fit().params)

Divide lcarat by its std dev:

Intercept        8.418208
ideal[T.True]    0.100013
lcarat           0.985628
dtype: float64


The original reg:

Intercept        8.418208
ideal[T.True]    0.100013
lcarat           1.696259
dtype: float64


**So a 1 standard deviation increase in  log𝑐𝑎𝑟𝑎𝑡  is associated with a 0.98% increase in price. Compared to  𝑖𝑑𝑒𝑎𝑙 , we can say that a reasonable variation in carats is associated with a price increase about 10 times the size.**

#### questions: 
- If x (the diamonds2 variable "x") goes up by:



In [10]:
#B. 1, how much does price go up?
sm_ols('lprice ~ X', 
       data=diamonds2).fit().summary()
    #can see that price goes up to $3,137
    


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 lprice   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                 3.768e+05
Date:                Thu, 16 Apr 2020   Prob (F-statistic):               0.00
Time:                        09:56:17   Log-Likelihood:                -4096.8
No. Observations:               53797   AIC:                             8200.
Df Residuals:                   53794   BIC:                             8226.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.602e+10   4.85e+10     -0.742      0.458   -1.31e+11    5.91e+10
X[0]        3.602e+10   4.85e+10      0.742      0.458   -5.91e+10    1.31e+11
X[1]           1.6819      0.002    868.025      0.000       1.678       1.686
==============================================================================
Omnibus:                      775.086   Durbin-Watson:                   1.210
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1334.320
Skew:                           0.106   Prob(JB):                    1.80e-290
Kurtosis:                       3.742   Cond. No.                     9.02e+13
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.45e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [15]:
#C. 1%, what % does price go up?
sm_ols('price ~ lcarat', 
       data=diamonds2).fit().summary()
#.729%

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                 1.447e+05
Date:                Thu, 16 Apr 2020   Prob (F-statistic):               0.00
Time:                        10:02:46   Log-Likelihood:            -4.8671e+05
No. Observations:               53797   AIC:                         9.734e+05
Df Residuals:                   53795   BIC:                         9.734e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   6215.9227     10.747    578.385      0.000    6194.858    6236.987
lcarat      5802.0412     15.251    380.443      0.000    5772.150    5831.933
==============================================================================
Omnibus:                    13143.502   Durbin-Watson:                   0.241
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33842.599
Skew:                           1.330   Prob(JB):                         0.00
Kurtosis:                       5.832   Cond. No.                         2.10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
# D. 1, what % does price go up? 
sm_ols('price ~ X', 
       data=diamonds2).fit().summary()
    #can see that it goes up .73%

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                 7.236e+04
Date:                Thu, 16 Apr 2020   Prob (F-statistic):               0.00
Time:                        10:04:36   Log-Likelihood:            -4.8671e+05
No. Observations:               53797   AIC:                         9.734e+05
Df Residuals:                   53794   BIC:                         9.735e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4.903e+14   3.82e+14      1.283      0.199   -2.59e+14    1.24e+15
X[0]       -4.903e+14   3.82e+14     -1.283      0.199   -1.24e+15    2.59e+14
X[1]        5802.2337     15.252    380.428      0.000    5772.340    5832.127
==============================================================================
Omnibus:                    13141.477   Durbin-Watson:                   0.241
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33832.979
Skew:                           1.330   Prob(JB):                         0.00
Kurtosis:                       5.832   Cond. No.                     9.02e+13
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.45e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [6]:
sm_ols('lprice ~ lcarat+ideal', data=diamonds2).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 lprice   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.936
Method:                 Least Squares   F-statistic:                 3.914e+05
Date:                Thu, 16 Apr 2020   Prob (F-statistic):               0.00
Time:                        09:55:47   Log-Likelihood:                -3136.4
No. Observations:               53797   AIC:                             6279.
Df Residuals:                   53794   BIC:                             6306.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         8.4182      0.002   5415.779      0.000       8.415       8.421
ideal[T.True]     0.1000      0.002     43.662      0.000       0.096       0.105
lcarat            1.6963      0.002    878.286      0.000       1.692       1.700
==============================================================================
Omnibus:                      794.680   Durbin-Watson:                   1.241
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1394.941
Skew:                           0.101   Prob(JB):                    1.24e-303
Kurtosis:                       3.763   Cond. No.                         2.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
diamonds2['lcarat'].describe()

count    53797.000000
mean        -0.398700
std          0.581060
min         -1.609438
25%         -0.916291
50%         -0.356675
75%          0.039221
max          0.912283
Name: lcarat, dtype: float64